In [98]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Libraries

In [99]:
from torchvision.transforms import Compose, ToTensor
from torchsummary import summary
import pickle
import pandas as pd
import plotly.express as px
from tqdm import tqdm

In [100]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import accuracy_metrics, forgetting_metrics, bwt_metrics, timing_metrics, cpu_usage_metrics, ram_usage_metrics

from torch.nn import CrossEntropyLoss
from torch.optim import SGD

## Custom Libraries

In [101]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.training import MWUN
from base_code.plugins import WeightStoragePlugin

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [102]:
scenario = PermutedMNIST(10, seed=1234, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [103]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [107]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True, trained_experience=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(experience=True, stream=True),
    ram_usage_metrics(experience=True, stream=True),
    loggers=[InteractiveLogger()]
)

## Plugin defitinitions

In [105]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [108]:
# model = MLP(n_classes=scenario.n_classes, n_channels=1, width=28, height=28)
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss()
strategy = MWUN(
    model,
    optimizer,
    criterion,
    eps=1e-5,
    train_epochs=5,
    train_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [109]:
results = list()

for experience in tqdm(train_stream):
    strategy.train(experience)

    # eval on the whole train stream
    metrics = strategy.eval(train_stream)
    results.append(metrics)

    # eval on test
    metrics = strategy.eval(test_stream)
    results.append(metrics)

  0%|          | 0/10 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 96.94it/s] 
Epoch 0 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 4.8386
100%|██████████| 469/469 [00:04<00:00, 98.40it/s] 
Epoch 1 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7664
100%|██████████| 469/469 [00:04<00:00, 98.01it/s] 
Epoch 2 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7853
100%|██████████| 469/469 [00:04<00:00, 98.68it/s] 
Epoch 3 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7530
100%|██████████| 469/469 [00:04<00:00, 99.28it/s] 
Epoch 4 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7243
-- >> End of training phase << --
-- >> Start of eval phase

 10%|█         | 1/10 [01:05<09:53, 65.91s/it]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 99.20it/s] 
Epoch 0 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7282
100%|██████████| 469/469 [00:04<00:00, 98.35it/s] 
Epoch 1 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7688
100%|██████████| 469/469 [00:04<00:00, 99.05it/s] 
Epoch 2 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7352
100%|██████████| 469/469 [00:04<00:00, 98.04it/s] 
Epoch 3 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7838
100%|██████████| 469/469 [00:04<00:00, 98.12it/s] 
Epoch 4 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7804
-- >> End of training phase << --
-- >> Start of eval phase

 20%|██        | 2/10 [02:12<08:51, 66.45s/it]

...

-- >> End of eval phase << --
	Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000 = 0.7275
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 106.0035
	MaxRAMUsage_Stream/eval_phase/test_stream/Task000 = 188.6875
	StreamBWT/eval_phase/test_stream = -0.0857
	StreamForgetting/eval_phase/test_stream = 0.0857
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.6038
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 95.38it/s] 
Epoch 0 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.9182
100%|██████████| 469/469 [00:04<00:00, 97.05it/s]
Epoch 1 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.8327
100%|██████████| 469/469 [00:04<00:00, 97.88it/s] 
Epoch 2 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7920
100%|██████████| 469/469 [00:04<00:00, 98.94it/

 90%|█████████ | 9/10 [10:24<01:08, 68.78s/it]

-- >> End of eval phase << --
	Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000 = 0.7213
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 105.9902
	MaxRAMUsage_Stream/eval_phase/test_stream/Task000 = 176.7344
	StreamBWT/eval_phase/test_stream = -0.0966
	StreamForgetting/eval_phase/test_stream = 0.0966
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.6630
-- >> Start of training phase << --
100%|██████████| 469/469 [00:04<00:00, 99.26it/s] 
Epoch 0 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7257
100%|██████████| 469/469 [00:04<00:00, 99.67it/s] 
Epoch 1 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7057
100%|██████████| 469/469 [00:04<00:00, 98.97it/s] 
Epoch 2 ended.
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 4.7391
100%|██████████| 469/469 [00:04<00:00, 98.78it

100%|██████████| 10/10 [11:30<00:00, 69.06s/it]

-- >> End of eval phase << --
	Accuracy_On_Trained_Experiences/eval_phase/test_stream/Task000 = 0.7149
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 106.0405
	MaxRAMUsage_Stream/eval_phase/test_stream/Task000 = 191.4219
	StreamBWT/eval_phase/test_stream = -0.1063
	StreamForgetting/eval_phase/test_stream = 0.1063
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.7149


# Get metrics

## Training Accuracies

In [112]:
training_accuracies: dict[int, list[float]] = dict()

for i in range(10):
    training_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/train_stream/Task000/Exp00{i}"
    ][1]

training_accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/train_stream/Task000"
][1]

## Evaluation Accuracies

In [113]:
accuracies: dict[int, list[float]] = dict()

for i in range(10):
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp00{i}"
    ][1]

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

## Forgetting measure

In [114]:
forgetting_measures: dict[int, list[float]] = dict()

for i in range(9):
    forgetting_measures[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp00{i}"
    ][1]
forgetting_measures["Overall"] = eval_plugin.get_all_metrics()[
    "StreamForgetting/eval_phase/test_stream"
][1]

## Backward Transfer

In [115]:
bwts: dict[int, list[float]] = dict()

for i in range(9):
    bwts[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp00{i}"
    ][1]
bwts["Overall"] = eval_plugin.get_all_metrics()[
    "StreamBWT/eval_phase/test_stream"
][1]

# Plotting metrics

## Traning accuracies

In [119]:
train_df = pd.DataFrame(training_accuracies)
train_df.index = range(10)

fig = px.line(train_df, x=train_df.index, y=train_df.columns, range_y=[0, 1], title="Training Accuracy vs Task")
fig.show()

## Evaluation accuracies per experience

In [120]:
acc_df = pd.DataFrame(accuracies)
acc_df.index = range(10)

fig = px.line(acc_df, x=acc_df.index, y=acc_df.columns, range_y=[0, 1], title="Test Accuracy vs Task")
fig.show()

## Forgetting measure / BWT

In [121]:
from copy import deepcopy
# transform forgetting_measures dict into df
# but first, we need to make sure that all lists have the same length
max_len = max(map(len, forgetting_measures.values()))
forgetting_measures_tmp = deepcopy(forgetting_measures)
forgetting_measures_tmp = {k: [None] * (max_len - len(v)) + v for k, v in forgetting_measures_tmp.items()}

forgetting_df = pd.DataFrame(forgetting_measures_tmp)
forgetting_df.index = range(10)

fig = px.line(forgetting_df, x=forgetting_df.index, y=forgetting_df.columns, title="Forgetting vs Task")
fig.show()

# Store metrics

In [122]:
pickle.dump(eval_plugin.get_all_metrics(), open(SAVED_METRICS_PATH / "mwun.pkl", "wb"))

# Store weights

In [124]:
pickle.dump(model_plugins[0].weights, open(SAVED_METRICS_PATH / "mwun_weights.pkl", "wb"))